In [1]:
import numpy as np
from model_functions import *
from Classification_models_LOSO_2Class import *

In [5]:
#uncomment to run and create the all_psds file. 

#subject_data, freqs, targets = load_data(duration=30,overlap=15,seg_length=4)
#save_psds(subject_data,freqs,targets,'all_psds.pickle')


In [2]:
subject_data, freqs, targets = load_psds('all_psds.pickle')
test_labels = align_test_labels(classes=['A','C'])
features, targets = remove_class(subject_data,targets,'F')
features, targets = remove_test(features,targets,test_labels)
rbps = [relative_band_power(features[i],freqs,freq_bands=[0.5,4.0,7.0,9.0,11.0,13.0,25.0,45.0]) 
        for i in range(len(features))]

### Logistic Regression

In [3]:
train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets)
train_metric_dict, val_metric_dict

({'acc': 0.9515982055236226,
  'sens': 0.9606124306518724,
  'spec': 0.9410549528541012,
  'f1': 0.9553536790378895},
 {'acc': 0.7595682041216879,
  'sens': 0.7888349514563107,
  'spec': 0.7253371185237757,
  'f1': 0.7796101949025488})

In [5]:
train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets, PCA_components = 133)
train_metric_dict, val_metric_dict

({'acc': 0.9516040469180803,
  'sens': 0.9606232662968099,
  'spec': 0.9410549528541012,
  'f1': 0.9553593077325115},
 {'acc': 0.7595682041216879,
  'sens': 0.7888349514563107,
  'spec': 0.7253371185237757,
  'f1': 0.7796101949025488})

In [6]:
PCA_components = [5, 10, 20, 40, 60, 80, 100, 133]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]

In [7]:
val_acc, train_acc

([0.7262021589793916,
  0.7340529931305201,
  0.7526987242394504,
  0.774288518155054,
  0.7392868825646058,
  0.7592410860320575,
  0.7579326136735361,
  0.7595682041216879],
 [0.7583765596523202,
  0.8085482966493761,
  0.8636910603299219,
  0.9034534324033834,
  0.9224613299686901,
  0.9415510070564045,
  0.9474099256974625,
  0.9516040469180803])

In [12]:
PCA_components = [30, 33, 35, 37, 40]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]

In [13]:
val_acc, train_acc

([0.7713444553483808,
  0.7811579980372915,
  0.7870461236506379,
  0.7808308799476611,
  0.7733071638861629],
 [0.8950126174120286,
  0.8986225991868779,
  0.901642600121501,
  0.9020748633113697,
  0.903476797981214])

In [14]:
train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets, PCA_components = 35)
train_metric_dict, val_metric_dict

({'acc': 0.9016951726716201,
  'sens': 0.9201846393897365,
  'spec': 0.8800694514853493,
  'f1': 0.9098477015958386},
 {'acc': 0.7841020608439647,
  'sens': 0.7979368932038835,
  'spec': 0.7679205110007097,
  'f1': 0.7993920972644377})

In [14]:
train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets, PCA_components = 35,reg_parameter=100.0)
train_metric_dict, val_metric_dict

({'acc': 0.9012629094817515,
  'sens': 0.9189927184466019,
  'spec': 0.8805257021190307,
  'f1': 0.9093804114149695},
 {'acc': 0.7870461236506379,
  'sens': 0.7985436893203883,
  'spec': 0.7735982966643009,
  'f1': 0.8017057569296375})

### k nearest neighbor classifier

In [4]:
train_metric_dict, val_metric_dict = kNN_cross(rbps, targets, n_neighbors = 3, PCA_components = 0)
train_metric_dict, val_metric_dict

({'acc': 0.977195196037198,
  'sens': 0.9817202669902912,
  'spec': 0.9719025651424515,
  'f1': 0.9789093816515944},
 {'acc': 0.671573438011122,
  'sens': 0.6644417475728155,
  'spec': 0.6799148332150461,
  'f1': 0.6856606136505948})

In [ ]:
train_metric_dict, val_metric_dict = kNN_cross(rbps, targets, n_neighbors = 3, PCA_components = 133)
train_metric_dict, val_metric_dict

In [5]:
PCA_components = [5, 10, 20, 40, 60, 80, 100, 133]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = kNN_cross(rbps, targets, n_neighbors = 3, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]

In [6]:
val_acc, train_acc

([0.6856395158652273,
  0.6719005561007524,
  0.6732090284592738,
  0.6735361465489041,
  0.6741903827281649,
  0.6735361465489041,
  0.6728819103696434,
  0.671573438011122],
 [0.8914493667928408,
  0.9308846207766718,
  0.9611138370951914,
  0.9741401467358288,
  0.9774522173933361,
  0.9764299733632412,
  0.977212720220571,
  0.977195196037198])

### Random Forest classifier

Here is a run of random Forest without doing PCA. As you can see in the definition of the method, PCA = 0, means we do not do PCA

In [8]:
train_metric_dict, val_metric_dict = RF_cross(rbps, targets, n_estimators = 25, 
                                              min_samples_split = 4, PCA_components = 0)
train_metric_dict, val_metric_dict

({'acc': 0.999129632225805,
  'sens': 0.9994690533980582,
  'spec': 0.9987326371286627,
  'f1': 0.999192967442465},
 {'acc': 0.7298004579653254,
  'sens': 0.7578883495145631,
  'spec': 0.6969481902058198,
  'f1': 0.7515042117930205})

Here is a run of random Forest with PCA. The maximum number of features is 133 here. 
As you can see, there is a significant reduction in validation set accuracy (from 75% to 71% )

In [ ]:
train_metric_dict, val_metric_dict = RF_cross(rbps, targets, n_estimators = 100, 
                                              min_samples_split = 16, PCA_components = 133)
train_metric_dict, val_metric_dict

In [ ]:
PCA_components = [5, 10, 20, 40, 60, 80, 100, 133]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = RF_cross(rbps, targets, n_estimators = 100, 
                                              min_samples_split = 16, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]

### SVM

In [10]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='linear',reg_parameter=0.1, PCA_components = 35)
train_metric_dict, val_metric_dict

({'acc': 0.9074898359736436,
  'sens': 0.9293732662968099,
  'spec': 0.881894454020075,
  'f1': 0.9154805552442401},
 {'acc': 0.8024206738632647,
  'sens': 0.8240291262135923,
  'spec': 0.7771469127040455,
  'f1': 0.8180722891566266})

In [12]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='poly',reg_parameter=1.0,degree=2, PCA_components = 35)
train_metric_dict, val_metric_dict

({'acc': 0.7810002803869339,
  'sens': 0.9781228328710125,
  'spec': 0.5504410422792254,
  'f1': 0.8280458104197149},
 {'acc': 0.5757278377494276,
  'sens': 0.8270631067961165,
  'spec': 0.2817601135557133,
  'f1': 0.6776037782749192})

In [13]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='poly',reg_parameter=10.0,degree=2, PCA_components = 35)
train_metric_dict, val_metric_dict

({'acc': 0.9252418337305481,
  'sens': 0.9684682732316228,
  'spec': 0.8746831592821657,
  'f1': 0.933188552575253},
 {'acc': 0.652273470722931,
  'sens': 0.7421116504854369,
  'spec': 0.5471965933286018,
  'f1': 0.6970646907950984})

In [14]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='poly',reg_parameter=100.0,degree=2, PCA_components = 35)
train_metric_dict, val_metric_dict

({'acc': 0.9673933361372027,
  'sens': 0.9839957524271845,
  'spec': 0.9479747541316029,
  'f1': 0.9701822610627978},
 {'acc': 0.6656853123977756,
  'sens': 0.7032766990291263,
  'spec': 0.6217175301632364,
  'f1': 0.6940119760479042})

In [15]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='rbf',reg_parameter=1.0, PCA_components = 35)
train_metric_dict, val_metric_dict

({'acc': 0.9390333660451423,
  'sens': 0.9639498092926491,
  'spec': 0.9098904998479165,
  'f1': 0.9445898523564045},
 {'acc': 0.7579326136735361,
  'sens': 0.7888349514563107,
  'spec': 0.7217885024840313,
  'f1': 0.7784431137724551})

In [16]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='rbf',reg_parameter=10.0, PCA_components = 35)
train_metric_dict, val_metric_dict

({'acc': 0.9664587130239731,
  'sens': 0.9756306345353676,
  'spec': 0.9557310149041873,
  'f1': 0.9690991281885696},
 {'acc': 0.7621851488387308,
  'sens': 0.7930825242718447,
  'spec': 0.7260468417317246,
  'f1': 0.7824004788985334})

In [17]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='rbf',reg_parameter=1.0, PCA_components = 0)
train_metric_dict, val_metric_dict

({'acc': 0.9542852469741576,
  'sens': 0.976118238557559,
  'spec': 0.9287488593734158,
  'f1': 0.9583710118408034},
 {'acc': 0.7500817795224076,
  'sens': 0.7858009708737864,
  'spec': 0.7083037615330021,
  'f1': 0.7722122838401908})

In [18]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='linear',reg_parameter=1.0,degree=2, PCA_components = 0)
train_metric_dict, val_metric_dict

({'acc': 0.9594607224636665,
  'sens': 0.9613492545076283,
  'spec': 0.9572518503497921,
  'f1': 0.9623607510494517},
 {'acc': 0.7360157016683022,
  'sens': 0.7682038834951457,
  'spec': 0.6983676366217175,
  'f1': 0.7583108715184187})

### MLP

In [22]:
train_metric_dict, val_metric_dict = mlp_cross(rbps, targets, PCA_components = 35, max_iter = 600, hidden_layer_sizes=(3,1))
train_metric_dict, val_metric_dict

c:\Users\Clark\anaconda3\envs\py_3_11\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Clark\anaconda3\envs\py_3_11\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Clark\anaconda3\envs\py_3_11\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Clark\anaconda3\envs\py_3_11\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Clark\anaco

({'acc': 0.9034359082200103,
  'sens': 0.9509578710124826,
  'spec': 0.8478530872959545,
  'f1': 0.913925698367655},
 {'acc': 0.7612037945698397,
  'sens': 0.8149271844660194,
  'spec': 0.6983676366217175,
  'f1': 0.7862997658079625})

In [13]:
train_metric_dict, val_metric_dict = mlp_cross(rbps, targets, PCA_components = 0, max_iter = 500, hidden_layer_sizes=(10,1))
train_metric_dict, val_metric_dict

c:\Users\Clark\anaconda3\envs\py_3_11\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Clark\anaconda3\envs\py_3_11\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Clark\anaconda3\envs\py_3_11\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Clark\anaconda3\envs\py_3_11\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


({'acc': 0.9890415439973831,
  'sens': 0.9856644417475728,
  'spec': 0.9929914833215047,
  'f1': 0.9897935867164293},
 {'acc': 0.7455021262675826,
  'sens': 0.7785194174757282,
  'spec': 0.7068843151171044,
  'f1': 0.7673444976076554})

In [21]:
train_metric_dict, val_metric_dict = mlp_cross(rbps, targets, PCA_components = 35, alpha = 1.0, max_iter = 500, hidden_layer_sizes=(100,1))
train_metric_dict, val_metric_dict

c:\Users\Clark\anaconda3\envs\py_3_11\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Clark\anaconda3\envs\py_3_11\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Clark\anaconda3\envs\py_3_11\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Clark\anaconda3\envs\py_3_11\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Clark\anaco

({'acc': 0.9609736436282069,
  'sens': 0.9680456830790569,
  'spec': 0.9527020176416912,
  'f1': 0.963956430494338},
 {'acc': 0.765129211645404,
  'sens': 0.787621359223301,
  'spec': 0.7388218594748048,
  'f1': 0.7833433916716959})